In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from matplotlib import pyplot as plt
import pickle
import sys
sys.path.append('./../')
sys.path.append('./../../')
import Portfolio_value as pv
# import import_ipynb

In [2]:
df =  pd.read_csv('./data/data.csv')
folder_name = 'dyna'
static_dyna = 'dyna'

In [3]:
p_val = np.load('./result/{}/p_val/p_val_0.07.npy'.format(folder_name), allow_pickle=True)
w_val = np.load('./result/{}/w_val/w_val_0.07.npy'.format(folder_name), allow_pickle=True)

p_val = p_val.reshape(1)
p_val = p_val[0]

w_val = w_val.reshape(1)
w_val = w_val[0]

In [4]:
bear_year = np.load('./result/year/bear_year.npy', allow_pickle=True).reshape(1)[0]


In [5]:
upper_years = [35]

In [6]:

# model_names = []
# for protection_level in [0.4]:
#     for multiplier in range(5,50,5):
#         model_names.append('TIPP_{}_{}'.format(int(protection_level*100), multiplier))
        

In [7]:
model_names = ['TIPP_100_5', 'TIPP_100_10', 'TIPP_100_15', 'TIPP_100_20', 'TIPP_100_25', 'TIPP_100_30', 'TIPP_100_35', 'TIPP_100_40', 'TIPP_100_45', 'TIPP_100_50']



In [8]:
# model_names = []

# for i in ['SP', 'SL']:
#     for j in ['95', '90']:
#             model_names.append(i+'_'+j)

In [9]:
bear_perform = {}
for model_name in model_names:
    bear_perform[model_name] = {
    'dep':[],
    'max':[],
    'under':[],
    'inc':[],
    'beq':[],
    'total':[]
}

In [10]:
# PI_Korea랑은 코드가 좀 다름

def get_metric_for_PI(retire_year, k, upper_year, model_name, withrate, const_or_dyna, df, init_wealth, p_val, w_val):
    # bequest
    
    p_amounts = p_val['{}_rate{}_{}'.format(model_name, withrate, const_or_dyna)][retire_year]
    bequest = p_amounts[12*upper_year]*init_wealth*(1-withrate)/100
    
    # withdraw_amount
    w_amounts = w_val['{}_rate{}_{}'.format(model_name, withrate, const_or_dyna)][retire_year]
    withdraw_amount = sum(w_amounts)
    
    # depletion time
    if np.argwhere(p_amounts == 0).size > 0 : # 고갈이 되면
        dep_time = np.argwhere(p_amounts == 0)[0][0]
    elif np.argwhere(p_amounts == 0).size == 0: # 고갈이 안되면
        dep_time = upper_year*12
        
    # max decline
    dec = np.min(p_amounts)/p_amounts[0]
    dec = dec - 1
    
    # underwater duration
    start_date = datetime(retire_year, 1, 1)
    end_date = start_date + relativedelta(years=upper_year)
    
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m')
    df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
    
    under_count = 0
    for i in range(upper_year*12):
        if p_amounts[i]*df.iloc[i]['CPI'] < p_amounts[0]*df.iloc[0]['CPI'] :
             under_count+= 1
        
    return bequest / init_wealth * 100, withdraw_amount / init_wealth * 100, dep_time, dec * 100, under_count

In [12]:
init_wealth = 500000
withdraw_rate = 0.07
k=0
const_or_dyna = 'dyna'
upper_year = 35

for model_name in model_names:
    print(model_name)
    for years in bear_year[upper_year]:
        for year in years:
            res = get_metric_for_PI(year, k, upper_year, model_name, withdraw_rate, const_or_dyna, df, init_wealth, p_val, w_val)
            if res[2] < upper_year*12 : #고갈 시점이 upper_year보다 작으면 (이미 고갈이 되면)
                bear_perform[model_name]['dep'].append(100)
            elif res[2] >= upper_year*12: # 고갈이 안되면
                bear_perform[model_name]['dep'].append(0)
            bear_perform[model_name]['max'].append(res[3])
            bear_perform[model_name]['under'].append(res[4])
            bear_perform[model_name]['inc'].append(res[1])
            bear_perform[model_name]['beq'].append(res[0])
            bear_perform[model_name]['total'].append(res[1] + res[0])


TIPP_100_5
TIPP_100_10
TIPP_100_15
TIPP_100_20
TIPP_100_25
TIPP_100_30
TIPP_100_35
TIPP_100_40
TIPP_100_45
TIPP_100_50


In [13]:
# Define all dictionaries and indices we want to calculate mean for
dicts = [bear_perform]
indices = model_names

# Use list comprehension to compute all means at once
# Store results in lists
a = [np.round(np.mean(d[i]['dep']),1) for i in indices for d in dicts]
b = [np.round(np.mean(d[i]['max']),1) for i in indices for d in dicts]
c = [np.round(np.mean(d[i]['under']),1) for i in indices for d in dicts]
d = [np.round(np.mean(d[i]['inc']),1) for i in indices for d in dicts]
e = [np.round(np.mean(d[i]['beq']),1) for i in indices for d in dicts]
f = [np.round(np.mean(d[i]['total']),1) for i in indices for d in dicts]

# If you need to access the results separately, you can do so by indexing the lists:
# For example, a1 corresponds to a_results[0], a2 corresponds to a_results[1], etc.

In [14]:

print(model_names)
print('Depletion Prob. (\%) & & ' + ' & '.join(['{}'] * len(a)).format(*a))
print('Max. decline (\%) & & ' + ' & '.join(['{}'] * len(b)).format(*b))
print('Underwater duration (month) & & ' + ' & '.join(['{}'] * len(c)).format(*c))
print('Total withdrawal (\%) & & ' + ' & '.join(['{}'] * len(d)).format(*d))
print('Bequest (\%) & & ' + ' & '.join(['{}'] * len(e)).format(*e))
print('Total withdrawal + Bequest (\%) & & ' + ' & '.join(['{}'] * len(f)).format(*f))





['TIPP_100_5', 'TIPP_100_10', 'TIPP_100_15', 'TIPP_100_20', 'TIPP_100_25', 'TIPP_100_30', 'TIPP_100_35', 'TIPP_100_40', 'TIPP_100_45', 'TIPP_100_50']
Depletion Prob. (\%) & & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0
Max. decline (\%) & & -100.0 & -100.0 & -100.0 & -100.0 & -100.0 & -100.0 & -100.0 & -100.0 & -100.0 & -100.0
Underwater duration (month) & & 402.6 & 402.6 & 402.6 & 402.6 & 402.6 & 402.6 & 402.6 & 402.6 & 402.6 & 402.6
Total withdrawal (\%) & & 99.6 & 99.6 & 99.6 & 99.6 & 99.6 & 99.6 & 99.6 & 99.6 & 99.6 & 99.6
Bequest (\%) & & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0
Total withdrawal + Bequest (\%) & & 99.6 & 99.6 & 99.6 & 99.6 & 99.6 & 99.6 & 99.6 & 99.6 & 99.6 & 99.6


In [ ]:
# 6% withrate

In [ ]:
['TIPP_100_5', 'TIPP_100_10', 'TIPP_100_15', 'TIPP_100_20', 'TIPP_100_25', 'TIPP_100_30', 'TIPP_100_35', 'TIPP_100_40', 'TIPP_100_45']
Depletion Prob. (\%) & & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0
Max. decline (\%) & & -100.0 & -100.0 & -100.0 & -100.0 & -100.0 & -100.0 & -100.0 & -100.0 & -100.0
Underwater duration (month) & & 398.1 & 398.1 & 398.1 & 398.1 & 398.1 & 398.1 & 398.1 & 398.1 & 398.1
Total withdrawal (\%) & & 100.7 & 100.7 & 100.7 & 100.7 & 100.7 & 100.7 & 100.7 & 100.7 & 100.7
Bequest (\%) & & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0
Total withdrawal + Bequest (\%) & & 100.7 & 100.7 & 100.7 & 100.7 & 100.7 & 100.7 & 100.7 & 100.7 & 100.7


In [ ]:
# 5% withrate

In [ ]:
['TIPP_60_5', 'TIPP_60_10', 'TIPP_60_15', 'TIPP_60_20', 'TIPP_60_25', 'TIPP_60_30', 'TIPP_60_35', 'TIPP_60_40', 'TIPP_60_45']
Depletion Prob. (\%) & & nan & nan & nan & nan & nan & nan & nan & nan & nan
Max. decline (\%) & & nan & nan & nan & nan & nan & nan & nan & nan & nan
Underwater duration (month) & & nan & nan & nan & nan & nan & nan & nan & nan & nan
Total withdrawal (\%) & & nan & nan & nan & nan & nan & nan & nan & nan & nan
Bequest (\%) & & nan & nan & nan & nan & nan & nan & nan & nan & nan
Total withdrawal + Bequest (\%) & & nan & nan & nan & nan & nan & nan & nan & nan & nan

In [ ]:
['TIPP_90_50', 'TIPP_90_60', 'TIPP_90_70', 'TIPP_90_80', 'TIPP_90_90']
Depletion Prob. (\%) & & 81.8 & 81.8 & 81.8 & 81.8 & 81.8
Max. decline (\%) & & -87.6 & -87.5 & -87.5 & -87.5 & -87.5
Underwater duration (month) & & 323.6 & 323.0 & 323.0 & 323.0 & 323.0
Total withdrawal (\%) & & 103.2 & 103.8 & 104.1 & 104.3 & 104.3
Bequest (\%) & & 27.9 & 28.2 & 28.2 & 28.1 & 27.9
Total withdrawal + Bequest (\%) & & 131.1 & 132.0 & 132.3 & 132.4 & 132.2

In [100]:
['TIPP_85_50', 'TIPP_85_60', 'TIPP_85_70', 'TIPP_85_80', 'TIPP_85_90']
Depletion Prob. (\%) & & 72.7 & 72.7 & 72.7 & 72.7 & 72.7
Max. decline (\%) & & -82.1 & -82.1 & -82.1 & -82.1 & -82.1
Underwater duration (month) & & 296.2 & 294.5 & 293.3 & 293.5 & 295.2
Total withdrawal (\%) & & 105.7 & 106.4 & 106.9 & 106.7 & 106.2
Bequest (\%) & & 45.6 & 46.0 & 46.3 & 46.4 & 46.4
Total withdrawal + Bequest (\%) & & 151.3 & 152.4 & 153.2 & 153.1 & 152.6

SyntaxError: invalid syntax (<ipython-input-100-a4a83c58c4b1>, line 2)

In [ ]:
['TIPP_80_50', 'TIPP_80_60', 'TIPP_80_70', 'TIPP_80_80', 'TIPP_80_90']
Depletion Prob. (\%) & & 59.1 & 59.1 & 59.1 & 59.1 & 59.1
Max. decline (\%) & & -77.1 & -77.5 & -77.9 & -77.9 & -77.9
Underwater duration (month) & & 275.5 & 276.1 & 277.5 & 277.4 & 277.4
Total withdrawal (\%) & & 114.5 & 113.9 & 113.1 & 113.1 & 113.1
Bequest (\%) & & 74.9 & 74.0 & 73.2 & 73.3 & 73.3
Total withdrawal + Bequest (\%) & & 189.4 & 187.9 & 186.4 & 186.4 & 186.4

In [ ]:
['TIPP_75_50', 'TIPP_75_60', 'TIPP_75_70', 'TIPP_75_80', 'TIPP_75_90']
Depletion Prob. (\%) & & 36.4 & 36.4 & 36.4 & 36.4 & 36.4
Max. decline (\%) & & -68.8 & -68.8 & -68.8 & -68.7 & -68.6
Underwater duration (month) & & 208.1 & 209.7 & 210.5 & 210.2 & 209.7
Total withdrawal (\%) & & 127.3 & 127.3 & 127.3 & 127.3 & 127.3
Bequest (\%) & & 169.8 & 168.6 & 168.2 & 168.7 & 169.0
Total withdrawal + Bequest (\%) & & 297.2 & 295.9 & 295.5 & 296.0 & 296.3

In [ ]:
['TIPP_70_50', 'TIPP_70_60', 'TIPP_70_70', 'TIPP_70_80', 'TIPP_70_90']
Depletion Prob. (\%) & & 45.5 & 50.0 & 50.0 & 50.0 & 50.0
Max. decline (\%) & & -71.9 & -72.0 & -72.0 & -72.0 & -72.0
Underwater duration (month) & & 220.5 & 220.8 & 220.8 & 220.8 & 220.8
Total withdrawal (\%) & & 127.0 & 126.7 & 126.7 & 126.7 & 126.7
Bequest (\%) & & 140.4 & 139.5 & 139.6 & 139.6 & 139.6
Total withdrawal + Bequest (\%) & & 267.4 & 266.2 & 266.3 & 266.3 & 266.3

In [47]:
['TIPP_100_5', 'TIPP_100_10', 'TIPP_100_15', 'TIPP_100_20', 'TIPP_100_25', 'TIPP_100_30', 'TIPP_100_35', 'TIPP_100_40', 'TIPP_100_45']
Depletion Prob. (\%) & & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0
Max. decline (\%) & & -100.0 & -100.0 & -100.0 & -100.0 & -100.0 & -100.0 & -100.0 & -100.0 & -100.0
Underwater duration (month) & & 375.2 & 375.2 & 375.2 & 375.2 & 375.2 & 375.2 & 375.2 & 375.2 & 375.2
Total withdrawal (\%) & & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0
Bequest (\%) & & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0
Total withdrawal + Bequest (\%) & & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0 & 100.0

SyntaxError: invalid syntax (<ipython-input-47-8af641492ae5>, line 2)

In [ ]:
['TIPP_95_5', 'TIPP_95_10', 'TIPP_95_15', 'TIPP_95_20', 'TIPP_95_25', 'TIPP_95_30', 'TIPP_95_35', 'TIPP_95_40', 'TIPP_95_45']
Depletion Prob. (\%) & & 95.5 & 90.9 & 86.4 & 86.4 & 86.4 & 86.4 & 86.4 & 81.8 & 81.8
Max. decline (\%) & & -99.6 & -99.1 & -96.7 & -94.2 & -93.7 & -93.7 & -93.5 & -92.8 & -92.5
Underwater duration (month) & & 367.6 & 356.6 & 327.5 & 314.0 & 308.6 & 316.7 & 316.5 & 315.0 & 314.5
Total withdrawal (\%) & & 108.1 & 109.4 & 106.3 & 105.1 & 104.3 & 100.5 & 100.5 & 100.5 & 100.3
Bequest (\%) & & 0.4 & 0.8 & 3.2 & 6.5 & 7.6 & 7.6 & 7.9 & 8.8 & 9.3
Total withdrawal + Bequest (\%) & & 108.5 & 110.3 & 109.6 & 111.6 & 111.9 & 108.1 & 108.4 & 109.3 & 109.6

In [ ]:
['TIPP_90_5', 'TIPP_90_10', 'TIPP_90_15', 'TIPP_90_20', 'TIPP_90_25', 'TIPP_90_30', 'TIPP_90_35', 'TIPP_90_40', 'TIPP_90_45']
Depletion Prob. (\%) & & 86.4 & 81.8 & 81.8 & 81.8 & 81.8 & 77.3 & 77.3 & 77.3 & 81.8
Max. decline (\%) & & -96.0 & -88.3 & -87.5 & -87.7 & -87.8 & -87.8 & -87.0 & -87.1 & -87.6
Underwater duration (month) & & 335.1 & 325.3 & 323.0 & 320.9 & 321.5 & 319.4 & 313.7 & 314.4 & 319.4
Total withdrawal (\%) & & 115.4 & 103.1 & 102.5 & 102.8 & 100.7 & 101.4 & 103.7 & 104.2 & 104.0
Bequest (\%) & & 4.1 & 16.9 & 23.2 & 27.2 & 27.2 & 27.7 & 30.1 & 30.1 & 28.7
Total withdrawal + Bequest (\%) & & 119.5 & 120.0 & 125.8 & 130.0 & 127.9 & 129.2 & 133.8 & 134.3 & 132.8

In [ ]:
['TIPP_85_5', 'TIPP_85_10', 'TIPP_85_15', 'TIPP_85_20', 'TIPP_85_25', 'TIPP_85_30', 'TIPP_85_35', 'TIPP_85_40', 'TIPP_85_45']
Depletion Prob. (\%) & & 77.3 & 72.7 & 68.2 & 68.2 & 68.2 & 72.7 & 72.7 & 72.7 & 72.7
Max. decline (\%) & & -87.9 & -83.2 & -81.3 & -80.5 & -81.5 & -82.1 & -82.2 & -82.1 & -82.1
Underwater duration (month) & & 307.5 & 300.4 & 293.5 & 290.6 & 298.9 & 299.8 & 298.6 & 297.9 & 297.2
Total withdrawal (\%) & & 117.5 & 109.8 & 109.0 & 107.3 & 106.2 & 104.5 & 104.6 & 105.2 & 105.5
Bequest (\%) & & 17.3 & 40.8 & 45.1 & 47.9 & 46.7 & 44.4 & 44.5 & 45.1 & 45.4
Total withdrawal + Bequest (\%) & & 134.9 & 150.6 & 154.1 & 155.2 & 152.8 & 148.8 & 149.2 & 150.3 & 150.9

In [ ]:
['TIPP_80_5', 'TIPP_80_10', 'TIPP_80_15', 'TIPP_80_20', 'TIPP_80_25', 'TIPP_80_30', 'TIPP_80_35', 'TIPP_80_40', 'TIPP_80_45']
Depletion Prob. (\%) & & 72.7 & 68.2 & 59.1 & 54.5 & 59.1 & 59.1 & 59.1 & 59.1 & 59.1
Max. decline (\%) & & -83.1 & -81.4 & -77.1 & -75.8 & -76.4 & -76.6 & -76.6 & -76.7 & -76.9
Underwater duration (month) & & 292.0 & 295.1 & 272.0 & 271.6 & 272.5 & 275.5 & 275.3 & 275.2 & 275.4
Total withdrawal (\%) & & 116.0 & 115.6 & 120.5 & 117.5 & 116.3 & 115.4 & 115.0 & 114.7 & 114.5
Bequest (\%) & & 41.2 & 54.5 & 70.1 & 76.5 & 76.4 & 76.0 & 76.0 & 75.8 & 75.4
Total withdrawal + Bequest (\%) & & 157.2 & 170.2 & 190.7 & 193.9 & 192.7 & 191.4 & 191.0 & 190.4 & 189.9

In [ ]:
['TIPP_75_5', 'TIPP_75_10', 'TIPP_75_15', 'TIPP_75_20', 'TIPP_75_25', 'TIPP_75_30', 'TIPP_75_35', 'TIPP_75_40', 'TIPP_75_45']
Depletion Prob. (\%) & & 59.1 & 45.5 & 45.5 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4
Max. decline (\%) & & -78.2 & -73.0 & -72.4 & -70.3 & -69.2 & -68.9 & -68.9 & -68.9 & -68.8
Underwater duration (month) & & 267.5 & 238.8 & 236.3 & 221.9 & 211.3 & 207.1 & 207.2 & 207.5 & 207.7
Total withdrawal (\%) & & 122.0 & 124.4 & 124.9 & 127.3 & 127.3 & 127.3 & 127.3 & 127.3 & 127.3
Bequest (\%) & & 71.8 & 112.1 & 131.5 & 154.8 & 165.2 & 172.5 & 171.8 & 171.2 & 170.5
Total withdrawal + Bequest (\%) & & 193.8 & 236.5 & 256.4 & 282.1 & 292.5 & 299.8 & 299.1 & 298.5 & 297.8

In [ ]:
['TIPP_70_5', 'TIPP_70_10', 'TIPP_70_15', 'TIPP_70_20', 'TIPP_70_25', 'TIPP_70_30', 'TIPP_70_35', 'TIPP_70_40', 'TIPP_70_45']
Depletion Prob. (\%) & & 45.5 & 40.9 & 40.9 & 40.9 & 40.9 & 40.9 & 40.9 & 45.5 & 45.5
Max. decline (\%) & & -72.2 & -68.5 & -70.3 & -70.2 & -70.9 & -71.1 & -71.3 & -71.6 & -71.7
Underwater duration (month) & & 235.2 & 211.1 & 224.0 & 219.8 & 220.0 & 220.2 & 220.3 & 220.5 & 220.5
Total withdrawal (\%) & & 126.0 & 128.1 & 126.6 & 127.6 & 127.6 & 127.4 & 127.4 & 127.2 & 127.2
Bequest (\%) & & 113.4 & 154.7 & 142.2 & 144.0 & 143.2 & 142.8 & 142.1 & 141.5 & 141.0
Total withdrawal + Bequest (\%) & & 239.4 & 282.8 & 268.8 & 271.6 & 270.7 & 270.1 & 269.5 & 268.7 & 268.1

In [ ]:
['TIPP_65_5', 'TIPP_65_10', 'TIPP_65_15', 'TIPP_65_20', 'TIPP_65_25', 'TIPP_65_30', 'TIPP_65_35', 'TIPP_65_40', 'TIPP_65_45']
Depletion Prob. (\%) & & 40.9 & 40.9 & 50.0 & 50.0 & 45.5 & 45.5 & 45.5 & 45.5 & 45.5
Max. decline (\%) & & -70.0 & -71.7 & -71.9 & -71.9 & -70.0 & -69.4 & -69.5 & -69.7 & -69.8
Underwater duration (month) & & 218.0 & 216.4 & 219.3 & 218.0 & 206.4 & 204.9 & 205.4 & 205.7 & 206.0
Total withdrawal (\%) & & 127.8 & 127.5 & 126.9 & 126.7 & 128.6 & 129.9 & 129.7 & 129.5 & 129.5
Bequest (\%) & & 144.8 & 143.0 & 142.3 & 142.9 & 149.9 & 152.3 & 151.8 & 151.2 & 150.7
Total withdrawal + Bequest (\%) & & 272.6 & 270.5 & 269.1 & 269.5 & 278.6 & 282.2 & 281.4 & 280.7 & 280.2

In [ ]:
['TIPP_60_5', 'TIPP_60_10', 'TIPP_60_15', 'TIPP_60_20', 'TIPP_60_25', 'TIPP_60_30', 'TIPP_60_35', 'TIPP_60_40', 'TIPP_60_45']
Depletion Prob. (\%) & & 36.4 & 45.5 & 45.5 & 45.5 & 45.5 & 45.5 & 45.5 & 45.5 & 45.5
Max. decline (\%) & & -70.6 & -70.2 & -70.4 & -70.4 & -70.4 & -70.4 & -70.4 & -70.4 & -70.4
Underwater duration (month) & & 211.5 & 207.5 & 208.9 & 208.9 & 208.9 & 208.9 & 208.9 & 208.9 & 208.9
Total withdrawal (\%) & & 128.4 & 129.0 & 129.0 & 129.0 & 129.0 & 129.0 & 129.0 & 129.0 & 129.0
Bequest (\%) & & 145.3 & 147.9 & 146.7 & 146.2 & 146.2 & 146.2 & 146.2 & 146.2 & 146.2
Total withdrawal + Bequest (\%) & & 273.8 & 276.9 & 275.7 & 275.3 & 275.3 & 275.3 & 275.3 & 275.3 & 275.3

In [ ]:
['TIPP_55_5', 'TIPP_55_10', 'TIPP_55_15', 'TIPP_55_20', 'TIPP_55_25', 'TIPP_55_30', 'TIPP_55_35', 'TIPP_55_40', 'TIPP_55_45']
Depletion Prob. (\%) & & 45.5 & 40.9 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4
Max. decline (\%) & & -70.0 & -68.2 & -66.8 & -66.9 & -67.1 & -67.2 & -67.2 & -67.2 & -67.2
Underwater duration (month) & & 206.5 & 198.6 & 191.4 & 188.2 & 189.8 & 192.5 & 193.5 & 194.1 & 195.0
Total withdrawal (\%) & & 129.5 & 132.2 & 133.1 & 134.0 & 133.5 & 132.9 & 132.7 & 132.7 & 132.7
Bequest (\%) & & 149.5 & 169.1 & 192.9 & 210.5 & 207.7 & 203.5 & 202.9 & 202.3 & 201.7
Total withdrawal + Bequest (\%) & & 279.0 & 301.3 & 326.0 & 344.6 & 341.2 & 336.4 & 335.6 & 335.0 & 334.4

In [ ]:
['TIPP_50_5', 'TIPP_50_10', 'TIPP_50_15', 'TIPP_50_20', 'TIPP_50_25', 'TIPP_50_30', 'TIPP_50_35', 'TIPP_50_40', 'TIPP_50_45']
Depletion Prob. (\%) & & 40.9 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4
Max. decline (\%) & & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5
Underwater duration (month) & & 199.6 & 197.6 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8
Total withdrawal (\%) & & 131.3 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2
Bequest (\%) & & 176.3 & 195.1 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9
Total withdrawal + Bequest (\%) & & 307.6 & 327.3 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1

In [ ]:
['TIPP_45_5', 'TIPP_45_10', 'TIPP_45_15', 'TIPP_45_20', 'TIPP_45_25', 'TIPP_45_30', 'TIPP_45_35', 'TIPP_45_40', 'TIPP_45_45']
Depletion Prob. (\%) & & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4
Max. decline (\%) & & -68.3 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5
Underwater duration (month) & & 197.5 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8
Total withdrawal (\%) & & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2
Bequest (\%) & & 195.6 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9
Total withdrawal + Bequest (\%) & & 327.8 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1

In [ ]:
['TIPP_40_5', 'TIPP_40_10', 'TIPP_40_15', 'TIPP_40_20', 'TIPP_40_25', 'TIPP_40_30', 'TIPP_40_35', 'TIPP_40_40', 'TIPP_40_45']
Depletion Prob. (\%) & & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4
Max. decline (\%) & & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5
Underwater duration (month) & & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8
Total withdrawal (\%) & & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2
Bequest (\%) & & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9
Total withdrawal + Bequest (\%) & & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1

In [13]:
['TIPP_95_1', 'TIPP_95_2', 'TIPP_95_3', 'TIPP_95_4', 'TIPP_95_5', 'TIPP_95_6', 'TIPP_95_7', 'TIPP_95_8', 'TIPP_95_9', 'TIPP_95_10', 'TIPP_95_11', 'TIPP_95_12', 'TIPP_95_13', 'TIPP_95_14', 'TIPP_95_15', 'TIPP_95_16', 'TIPP_95_17', 'TIPP_95_18', 'TIPP_95_19', 'TIPP_95_20']
Depletion Prob. (\%) & & 100.0 & 100.0 & 100.0 & 95.5 & 95.5 & 95.5 & 95.5 & 95.5 & 90.9 & 90.9 & 90.9 & 86.4 & 86.4 & 86.4 & 86.4 & 86.4 & 90.9 & 90.9 & 86.4 & 86.4
Max. decline (\%) & & -100.0 & -100.0 & -100.0 & -99.9 & -99.6 & -99.5 & -99.4 & -99.5 & -99.5 & -99.1 & -98.8 & -98.3 & -97.7 & -97.2 & -96.7 & -96.3 & -96.1 & -95.3 & -94.7 & -94.2
Underwater duration (month) & & 374.2 & 372.9 & 370.6 & 369.5 & 367.6 & 365.9 & 365.8 & 367.1 & 364.2 & 356.6 & 348.3 & 340.0 & 334.5 & 330.3 & 327.5 & 325.5 & 321.1 & 319.0 & 316.6 & 314.0
Total withdrawal (\%) & & 102.4 & 104.9 & 106.7 & 107.4 & 108.1 & 108.8 & 109.2 & 109.0 & 109.6 & 109.4 & 109.4 & 108.1 & 107.5 & 106.7 & 106.3 & 107.4 & 107.7 & 106.9 & 106.0 & 105.1
Bequest (\%) & & 0.0 & 0.0 & 0.0 & 0.1 & 0.4 & 0.5 & 0.6 & 0.5 & 0.5 & 0.8 & 1.1 & 1.7 & 2.2 & 2.7 & 3.2 & 3.6 & 3.9 & 4.6 & 5.5 & 6.5
Total withdrawal + Bequest (\%) & & 102.4 & 104.9 & 106.7 & 107.5 & 108.5 & 109.4 & 109.8 & 109.4 & 110.1 & 110.3 & 110.5 & 109.7 & 109.7 & 109.5 & 109.6 & 111.0 & 111.6 & 111.6 & 111.5 & 111.6

SyntaxError: invalid syntax (<ipython-input-13-9a8c225b2032>, line 2)

In [ ]:
['TIPP_90_1', 'TIPP_90_2', 'TIPP_90_3', 'TIPP_90_4', 'TIPP_90_5', 'TIPP_90_6', 'TIPP_90_7', 'TIPP_90_8', 'TIPP_90_9', 'TIPP_90_10', 'TIPP_90_11', 'TIPP_90_12', 'TIPP_90_13', 'TIPP_90_14', 'TIPP_90_15', 'TIPP_90_16', 'TIPP_90_17', 'TIPP_90_18', 'TIPP_90_19', 'TIPP_90_20']
Depletion Prob. (\%) & & 100.0 & 95.5 & 95.5 & 90.9 & 86.4 & 86.4 & 81.8 & 81.8 & 81.8 & 81.8 & 81.8 & 81.8 & 81.8 & 81.8 & 81.8 & 81.8 & 81.8 & 81.8 & 81.8 & 81.8
Max. decline (\%) & & -100.0 & -99.6 & -98.8 & -97.8 & -96.0 & -93.6 & -91.4 & -89.6 & -89.0 & -88.3 & -88.1 & -87.8 & -87.6 & -87.5 & -87.5 & -87.5 & -87.5 & -87.6 & -87.6 & -87.7
Underwater duration (month) & & 372.2 & 367.8 & 363.1 & 353.5 & 335.1 & 328.4 & 323.4 & 325.5 & 325.3 & 325.3 & 324.9 & 324.4 & 323.9 & 323.7 & 323.0 & 322.1 & 321.0 & 320.8 & 320.9 & 320.9
Total withdrawal (\%) & & 105.8 & 109.9 & 113.3 & 115.1 & 115.4 & 113.6 & 111.7 & 108.3 & 105.3 & 103.1 & 102.7 & 102.0 & 100.8 & 102.1 & 102.5 & 102.9 & 103.4 & 103.4 & 103.4 & 102.8
Bequest (\%) & & 0.0 & 0.4 & 1.2 & 2.2 & 4.1 & 6.6 & 8.9 & 11.2 & 13.9 & 16.9 & 17.9 & 19.1 & 20.2 & 21.6 & 23.2 & 25.0 & 26.4 & 27.2 & 27.2 & 27.2
Total withdrawal + Bequest (\%) & & 105.8 & 110.3 & 114.5 & 117.4 & 119.5 & 120.1 & 120.6 & 119.5 & 119.3 & 120.0 & 120.6 & 121.0 & 121.1 & 123.7 & 125.8 & 127.9 & 129.8 & 130.6 & 130.5 & 130.0

In [ ]:
['TIPP_85_1', 'TIPP_85_2', 'TIPP_85_3', 'TIPP_85_4', 'TIPP_85_5', 'TIPP_85_6', 'TIPP_85_7', 'TIPP_85_8', 'TIPP_85_9', 'TIPP_85_10', 'TIPP_85_11', 'TIPP_85_12', 'TIPP_85_13', 'TIPP_85_14', 'TIPP_85_15', 'TIPP_85_16', 'TIPP_85_17', 'TIPP_85_18', 'TIPP_85_19', 'TIPP_85_20']
Depletion Prob. (\%) & & 95.5 & 95.5 & 86.4 & 81.8 & 77.3 & 77.3 & 77.3 & 72.7 & 72.7 & 72.7 & 72.7 & 68.2 & 68.2 & 68.2 & 68.2 & 68.2 & 68.2 & 68.2 & 68.2 & 68.2
Max. decline (\%) & & -100.0 & -98.6 & -95.8 & -91.3 & -87.9 & -86.6 & -86.4 & -85.3 & -83.9 & -83.2 & -82.9 & -82.4 & -81.8 & -81.3 & -81.3 & -81.4 & -81.3 & -81.0 & -80.7 & -80.5
Underwater duration (month) & & 369.5 & 358.0 & 335.0 & 319.0 & 307.5 & 308.4 & 310.9 & 310.7 & 304.8 & 300.4 & 299.9 & 299.3 & 297.3 & 293.1 & 293.5 & 295.8 & 295.9 & 293.3 & 291.8 & 290.6
Total withdrawal (\%) & & 108.9 & 116.3 & 121.2 & 120.1 & 117.5 & 112.6 & 108.1 & 108.4 & 108.9 & 109.8 & 110.5 & 111.2 & 110.9 & 110.3 & 109.0 & 108.3 & 107.9 & 107.5 & 107.3 & 107.3
Bequest (\%) & & 0.0 & 1.5 & 4.4 & 10.2 & 17.3 & 24.4 & 29.8 & 35.3 & 39.1 & 40.8 & 42.6 & 44.0 & 44.7 & 45.4 & 45.1 & 44.9 & 45.1 & 45.9 & 46.8 & 47.9
Total withdrawal + Bequest (\%) & & 108.9 & 117.7 & 125.6 & 130.3 & 134.9 & 137.0 & 137.9 & 143.7 & 148.0 & 150.6 & 153.1 & 155.2 & 155.5 & 155.7 & 154.1 & 153.2 & 153.0 & 153.4 & 154.2 & 155.2

In [ ]:
['TIPP_80_1', 'TIPP_80_2', 'TIPP_80_3', 'TIPP_80_4', 'TIPP_80_5', 'TIPP_80_6', 'TIPP_80_7', 'TIPP_80_8', 'TIPP_80_9', 'TIPP_80_10', 'TIPP_80_11', 'TIPP_80_12', 'TIPP_80_13', 'TIPP_80_14', 'TIPP_80_15', 'TIPP_80_16', 'TIPP_80_17', 'TIPP_80_18', 'TIPP_80_19', 'TIPP_80_20']
Depletion Prob. (\%) & & 95.5 & 86.4 & 63.6 & 68.2 & 72.7 & 63.6 & 59.1 & 54.5 & 63.6 & 68.2 & 63.6 & 63.6 & 59.1 & 59.1 & 59.1 & 59.1 & 54.5 & 54.5 & 54.5 & 54.5
Max. decline (\%) & & -99.5 & -96.4 & -88.9 & -84.5 & -83.1 & -82.6 & -80.6 & -79.2 & -80.7 & -81.4 & -80.9 & -79.7 & -78.0 & -77.5 & -77.1 & -76.7 & -76.0 & -75.5 & -75.5 & -75.8
Underwater duration (month) & & 365.0 & 339.6 & 307.7 & 294.5 & 292.0 & 290.1 & 287.8 & 280.3 & 292.6 & 295.1 & 289.9 & 286.7 & 284.5 & 276.5 & 272.0 & 271.2 & 270.5 & 271.0 & 271.6 & 271.6
Total withdrawal (\%) & & 112.0 & 123.1 & 126.6 & 122.5 & 116.0 & 118.2 & 120.0 & 120.8 & 117.8 & 115.6 & 117.2 & 118.7 & 119.6 & 120.0 & 120.5 & 120.8 & 120.2 & 119.3 & 117.5 & 117.5
Bequest (\%) & & 0.5 & 3.7 & 13.4 & 26.5 & 41.2 & 50.2 & 56.1 & 59.9 & 56.7 & 54.5 & 56.4 & 58.9 & 63.1 & 66.5 & 70.1 & 73.6 & 76.5 & 78.0 & 76.9 & 76.5
Total withdrawal + Bequest (\%) & & 112.5 & 126.9 & 140.0 & 149.0 & 157.2 & 168.4 & 176.1 & 180.7 & 174.5 & 170.2 & 173.6 & 177.6 & 182.7 & 186.5 & 190.7 & 194.3 & 196.7 & 197.3 & 194.3 & 193.9

In [ ]:
['TIPP_75_1', 'TIPP_75_2', 'TIPP_75_3', 'TIPP_75_4', 'TIPP_75_5', 'TIPP_75_6', 'TIPP_75_7', 'TIPP_75_8', 'TIPP_75_9', 'TIPP_75_10', 'TIPP_75_11', 'TIPP_75_12', 'TIPP_75_13', 'TIPP_75_14', 'TIPP_75_15', 'TIPP_75_16', 'TIPP_75_17', 'TIPP_75_18', 'TIPP_75_19', 'TIPP_75_20']
Depletion Prob. (\%) & & 95.5 & 72.7 & 59.1 & 63.6 & 59.1 & 54.5 & 50.0 & 50.0 & 50.0 & 45.5 & 45.5 & 45.5 & 45.5 & 45.5 & 45.5 & 45.5 & 45.5 & 36.4 & 36.4 & 36.4
Max. decline (\%) & & -98.9 & -91.3 & -80.3 & -80.3 & -78.2 & -76.6 & -74.8 & -73.5 & -73.0 & -73.0 & -72.9 & -73.0 & -72.8 & -72.6 & -72.4 & -72.1 & -71.9 & -71.1 & -70.5 & -70.3
Underwater duration (month) & & 356.7 & 312.7 & 272.3 & 276.2 & 267.5 & 263.9 & 255.9 & 243.5 & 239.8 & 238.8 & 239.1 & 239.9 & 239.4 & 237.8 & 236.3 & 233.5 & 229.8 & 226.1 & 223.4 & 221.9
Total withdrawal (\%) & & 115.7 & 128.0 & 129.2 & 120.7 & 122.0 & 122.4 & 123.0 & 124.0 & 124.6 & 124.4 & 124.1 & 123.9 & 124.1 & 124.6 & 124.9 & 125.8 & 126.8 & 127.0 & 127.0 & 127.3
Bequest (\%) & & 1.1 & 9.9 & 32.8 & 55.2 & 71.8 & 76.9 & 86.4 & 98.7 & 106.1 & 112.1 & 115.7 & 118.2 & 122.0 & 126.8 & 131.5 & 136.7 & 142.2 & 148.7 & 153.2 & 154.8
Total withdrawal + Bequest (\%) & & 116.8 & 137.9 & 162.0 & 175.9 & 193.8 & 199.3 & 209.3 & 222.6 & 230.8 & 236.5 & 239.8 & 242.1 & 246.0 & 251.4 & 256.4 & 262.5 & 269.0 & 275.7 & 280.1 & 282.1

In [ ]:
['TIPP_70_1', 'TIPP_70_2', 'TIPP_70_3', 'TIPP_70_4', 'TIPP_70_5', 'TIPP_70_6', 'TIPP_70_7', 'TIPP_70_8', 'TIPP_70_9', 'TIPP_70_10', 'TIPP_70_11', 'TIPP_70_12', 'TIPP_70_13', 'TIPP_70_14', 'TIPP_70_15', 'TIPP_70_16', 'TIPP_70_17', 'TIPP_70_18', 'TIPP_70_19', 'TIPP_70_20']
Depletion Prob. (\%) & & 95.5 & 59.1 & 45.5 & 50.0 & 45.5 & 40.9 & 40.9 & 40.9 & 40.9 & 40.9 & 40.9 & 36.4 & 40.9 & 40.9 & 40.9 & 40.9 & 40.9 & 40.9 & 40.9 & 40.9
Max. decline (\%) & & -98.3 & -82.4 & -75.5 & -74.7 & -72.2 & -70.3 & -69.2 & -68.7 & -68.6 & -68.5 & -68.8 & -69.7 & -70.0 & -70.1 & -70.3 & -70.1 & -70.0 & -69.9 & -70.0 & -70.2
Underwater duration (month) & & 350.6 & 269.5 & 250.5 & 247.1 & 235.2 & 220.7 & 212.5 & 209.9 & 211.7 & 211.1 & 215.5 & 216.9 & 218.2 & 218.2 & 224.0 & 222.8 & 221.3 & 219.6 & 219.7 & 219.8
Total withdrawal (\%) & & 119.7 & 131.2 & 128.2 & 125.7 & 126.0 & 128.0 & 127.6 & 127.8 & 127.8 & 128.1 & 128.1 & 127.7 & 127.6 & 127.1 & 126.6 & 126.9 & 127.0 & 127.6 & 127.6 & 127.6
Bequest (\%) & & 1.7 & 22.1 & 62.1 & 95.1 & 113.4 & 138.8 & 148.4 & 152.6 & 153.2 & 154.7 & 151.8 & 148.3 & 146.2 & 144.4 & 142.2 & 143.0 & 143.8 & 144.5 & 144.3 & 144.0
Total withdrawal + Bequest (\%) & & 121.4 & 153.4 & 190.3 & 220.8 & 239.4 & 266.8 & 276.0 & 280.4 & 281.0 & 282.8 & 279.9 & 276.1 & 273.8 & 271.5 & 268.8 & 269.9 & 270.8 & 272.1 & 271.9 & 271.6

In [ ]:
['TIPP_65_1', 'TIPP_65_2', 'TIPP_65_3', 'TIPP_65_4', 'TIPP_65_5', 'TIPP_65_6', 'TIPP_65_7', 'TIPP_65_8', 'TIPP_65_9', 'TIPP_65_10', 'TIPP_65_11', 'TIPP_65_12', 'TIPP_65_13', 'TIPP_65_14', 'TIPP_65_15', 'TIPP_65_16', 'TIPP_65_17', 'TIPP_65_18', 'TIPP_65_19', 'TIPP_65_20']
Depletion Prob. (\%) & & 81.8 & 45.5 & 50.0 & 50.0 & 40.9 & 40.9 & 40.9 & 40.9 & 40.9 & 40.9 & 45.5 & 45.5 & 45.5 & 50.0 & 50.0 & 50.0 & 50.0 & 50.0 & 50.0 & 50.0
Max. decline (\%) & & -97.0 & -76.3 & -73.7 & -72.0 & -70.0 & -70.0 & -70.7 & -71.1 & -71.5 & -71.7 & -71.8 & -71.8 & -71.8 & -71.9 & -71.9 & -72.0 & -72.1 & -72.2 & -72.0 & -71.9
Underwater duration (month) & & 340.5 & 253.1 & 240.5 & 227.8 & 218.0 & 218.6 & 221.6 & 217.8 & 216.7 & 216.4 & 216.5 & 216.8 & 217.4 & 218.1 & 219.3 & 220.5 & 221.5 & 222.2 & 220.7 & 218.0
Total withdrawal (\%) & & 123.6 & 132.3 & 127.5 & 127.8 & 127.8 & 127.3 & 127.0 & 127.7 & 127.7 & 127.5 & 127.4 & 127.0 & 127.0 & 126.9 & 126.9 & 126.5 & 126.5 & 126.1 & 126.5 & 126.7
Bequest (\%) & & 3.1 & 40.0 & 99.5 & 126.6 & 144.8 & 145.5 & 142.7 & 143.5 & 143.2 & 143.0 & 142.7 & 142.6 & 142.7 & 142.6 & 142.3 & 141.8 & 141.3 & 141.0 & 141.9 & 142.9
Total withdrawal + Bequest (\%) & & 126.7 & 172.3 & 227.0 & 254.4 & 272.6 & 272.8 & 269.7 & 271.2 & 270.9 & 270.5 & 270.1 & 269.7 & 269.8 & 269.5 & 269.1 & 268.3 & 267.8 & 267.2 & 268.4 & 269.5

In [ ]:
['TIPP_60_1', 'TIPP_60_2', 'TIPP_60_3', 'TIPP_60_4', 'TIPP_60_5', 'TIPP_60_6', 'TIPP_60_7', 'TIPP_60_8', 'TIPP_60_9', 'TIPP_60_10', 'TIPP_60_11', 'TIPP_60_12', 'TIPP_60_13', 'TIPP_60_14', 'TIPP_60_15', 'TIPP_60_16', 'TIPP_60_17', 'TIPP_60_18', 'TIPP_60_19', 'TIPP_60_20']
Depletion Prob. (\%) & & 77.3 & 45.5 & 50.0 & 40.9 & 36.4 & 45.5 & 45.5 & 45.5 & 45.5 & 45.5 & 45.5 & 45.5 & 45.5 & 45.5 & 45.5 & 45.5 & 45.5 & 45.5 & 45.5 & 45.5
Max. decline (\%) & & -93.9 & -71.6 & -71.9 & -70.1 & -70.6 & -70.6 & -69.7 & -69.9 & -70.1 & -70.2 & -70.4 & -70.4 & -70.4 & -70.4 & -70.4 & -70.4 & -70.4 & -70.4 & -70.4 & -70.4
Underwater duration (month) & & 325.3 & 229.0 & 226.6 & 218.5 & 211.5 & 209.7 & 205.7 & 206.2 & 206.9 & 207.5 & 208.8 & 208.9 & 208.9 & 208.9 & 208.9 & 208.9 & 208.9 & 208.9 & 208.9 & 208.9
Total withdrawal (\%) & & 126.9 & 132.1 & 128.2 & 127.8 & 128.4 & 128.7 & 129.9 & 129.2 & 129.4 & 129.0 & 129.0 & 129.0 & 129.0 & 129.0 & 129.0 & 129.0 & 129.0 & 129.0 & 129.0 & 129.0
Bequest (\%) & & 6.3 & 64.5 & 126.9 & 143.0 & 145.3 & 146.5 & 149.9 & 149.4 & 148.6 & 147.9 & 147.2 & 147.1 & 147.0 & 146.8 & 146.7 & 146.6 & 146.4 & 146.3 & 146.2 & 146.2
Total withdrawal + Bequest (\%) & & 133.2 & 196.6 & 255.1 & 270.8 & 273.8 & 275.1 & 279.8 & 278.5 & 278.0 & 276.9 & 276.2 & 276.1 & 276.0 & 275.8 & 275.7 & 275.6 & 275.4 & 275.3 & 275.3 & 275.3

In [ ]:
['TIPP_55_1', 'TIPP_55_2', 'TIPP_55_3', 'TIPP_55_4', 'TIPP_55_5', 'TIPP_55_6', 'TIPP_55_7', 'TIPP_55_8', 'TIPP_55_9', 'TIPP_55_10', 'TIPP_55_11', 'TIPP_55_12', 'TIPP_55_13', 'TIPP_55_14', 'TIPP_55_15', 'TIPP_55_16', 'TIPP_55_17', 'TIPP_55_18', 'TIPP_55_19', 'TIPP_55_20']
Depletion Prob. (\%) & & 68.2 & 45.5 & 40.9 & 40.9 & 45.5 & 40.9 & 40.9 & 40.9 & 40.9 & 40.9 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4
Max. decline (\%) & & -89.9 & -70.7 & -69.7 & -70.4 & -70.0 & -69.0 & -68.8 & -68.5 & -68.4 & -68.2 & -67.6 & -67.1 & -67.0 & -66.9 & -66.8 & -66.8 & -66.9 & -66.9 & -66.9 & -66.9
Underwater duration (month) & & 295.2 & 223.7 & 216.1 & 209.2 & 206.5 & 207.4 & 203.7 & 200.6 & 200.0 & 198.6 & 197.5 & 197.0 & 196.2 & 193.2 & 191.4 & 190.2 & 189.4 & 188.7 & 188.5 & 188.2
Total withdrawal (\%) & & 130.1 & 131.2 & 129.4 & 127.6 & 129.5 & 129.9 & 130.1 & 130.6 & 131.5 & 132.2 & 132.4 & 132.6 & 132.7 & 132.9 & 133.1 & 133.3 & 133.5 & 133.9 & 134.0 & 134.0
Bequest (\%) & & 11.0 & 94.3 & 150.4 & 145.7 & 149.5 & 153.0 & 156.6 & 160.7 & 164.9 & 169.1 & 174.0 & 178.9 & 183.6 & 188.3 & 192.9 & 197.4 & 201.8 & 206.1 & 209.2 & 210.5
Total withdrawal + Bequest (\%) & & 141.1 & 225.5 & 279.8 & 273.3 & 279.0 & 282.9 & 286.7 & 291.3 & 296.4 & 301.3 & 306.4 & 311.4 & 316.4 & 321.2 & 326.0 & 330.7 & 335.3 & 340.0 & 343.2 & 344.6

In [ ]:
['TIPP_50_1', 'TIPP_50_2', 'TIPP_50_3', 'TIPP_50_4', 'TIPP_50_5', 'TIPP_50_6', 'TIPP_50_7', 'TIPP_50_8', 'TIPP_50_9', 'TIPP_50_10', 'TIPP_50_11', 'TIPP_50_12', 'TIPP_50_13', 'TIPP_50_14', 'TIPP_50_15', 'TIPP_50_16', 'TIPP_50_17', 'TIPP_50_18', 'TIPP_50_19', 'TIPP_50_20']
Depletion Prob. (\%) & & 63.6 & 50.0 & 36.4 & 40.9 & 40.9 & 40.9 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4
Max. decline (\%) & & -85.5 & -71.4 & -68.8 & -68.6 & -68.5 & -68.5 & -67.8 & -67.7 & -68.1 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5
Underwater duration (month) & & 282.9 & 221.5 & 203.6 & 200.4 & 199.6 & 198.2 & 197.1 & 197.0 & 197.3 & 197.6 & 197.7 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8
Total withdrawal (\%) & & 132.1 & 130.3 & 129.9 & 130.4 & 131.3 & 132.2 & 132.4 & 132.4 & 132.4 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2
Bequest (\%) & & 17.5 & 130.1 & 161.2 & 167.5 & 176.3 & 185.2 & 194.9 & 197.5 & 196.3 & 195.1 & 195.0 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9
Total withdrawal + Bequest (\%) & & 149.7 & 260.4 & 291.1 & 297.9 & 307.6 & 317.3 & 327.3 & 329.9 & 328.7 & 327.3 & 327.2 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1

In [ ]:
['TIPP_45_1', 'TIPP_45_2', 'TIPP_45_3', 'TIPP_45_4', 'TIPP_45_5', 'TIPP_45_6', 'TIPP_45_7', 'TIPP_45_8', 'TIPP_45_9', 'TIPP_45_10', 'TIPP_45_11', 'TIPP_45_12', 'TIPP_45_13', 'TIPP_45_14', 'TIPP_45_15', 'TIPP_45_16', 'TIPP_45_17', 'TIPP_45_18', 'TIPP_45_19', 'TIPP_45_20']
Depletion Prob. (\%) & & 59.1 & 40.9 & 40.9 & 40.9 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4 & 36.4
Max. decline (\%) & & -80.5 & -70.4 & -68.4 & -68.6 & -68.3 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5 & -68.5
Underwater duration (month) & & 258.7 & 215.9 & 199.1 & 198.6 & 197.5 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8 & 197.8
Total withdrawal (\%) & & 133.6 & 130.1 & 130.6 & 132.0 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2 & 132.2
Bequest (\%) & & 25.6 & 153.9 & 174.9 & 188.6 & 195.6 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9 & 194.9
Total withdrawal + Bequest (\%) & & 159.2 & 283.9 & 305.5 & 320.6 & 327.8 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1 & 327.1